In [16]:
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
import re
import pandas as pd
from datetime import datetime

In [17]:
now = datetime.now()
date = now.strftime("%Y")
time = str(date)

In [18]:
url1 = "https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=list&&articleLimit=500&article.offset=0" #학사안내
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}

html = requests.get(url1, headers=headers)
soup = BeautifulSoup(html.content, 'html.parser')
data = soup.find('div', {'class': 'board-list01'})

table = data.find('table')
p = parser_functions.make2d(table)
#판다스 데이터프레임으로 변환
df = pd.DataFrame(p[1:], columns=p[0])


In [19]:
#번호 없는 행 개수 세기
count = 0
for i in df['번호']:
    if i == '':
        count += 1
#count만큼 행 삭제
df = df.drop(df.index[0:count])

#인덱스 삭제
df = df.reset_index(drop=True)
#조회수 삭제
df = df.drop(['조회'], axis=1)
df = df.drop(['첨부'], axis=1)
df = df.drop(['작성자'], axis=1)

In [20]:
#제목만 남기고 나머지 삭제
for i in df['제목']:
    pretreatment = i.split('\n')
    clean_text = pretreatment[0]
    #판다스 제목행 수정
    df['제목'] = df['제목'].replace(i, clean_text)



In [21]:
#title left 클레스 안의 href 가져오기
title = data.find_all('td', {'class': 'title left'})
for i in range(len(title)):
    title[i] = title[i].find('a')
    title[i] = title[i]['href']
title = title[count:]




In [22]:
title_url = []
for i in title:
    url = "https://www.kumoh.ac.kr/ko/sub06_01_01_01.do"
    title_url.append(url + i)
    print(url + i)


https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=469384&article.offset=0&articleLimit=500
https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=469383&article.offset=0&articleLimit=500
https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=469304&article.offset=0&articleLimit=500
https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=469285&article.offset=0&articleLimit=500
https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=469225&article.offset=0&articleLimit=500
https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=469143&article.offset=0&articleLimit=500
https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=469101&article.offset=0&articleLimit=500
https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=468977&article.offset=0&articleLimit=500
https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=468876&article.offset=0&articleLimit=500
https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?m

In [23]:
#link 열 추가 후 title리스트에 있는 href값 넣기

df['link'] = title_url
df.to_csv('/Users/dryoon04/Documents/GitHub/university-project/discord_chatbot/data/school_notice.csv', index=False)

In [25]:
import openai
import os
import langchain
from langchain.document_loaders.csv_loader import CSVLoader
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key_path = 'discord_chatbot/gptkey.txt'

loader = CSVLoader(file_path="./data/school_notice.csv")

data = loader.load()



[Document(page_content='번호: 5830\n카테고리: [장학]\n제목: 2023년도 영·호남 상생 유공 장학생 선발 안내\n등록일: 2023-11-15\nlink: https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=469384&article.offset=0&articleLimit=500', metadata={'source': './data/school_notice.csv', 'row': 0}),
 Document(page_content='번호: 5829\n카테고리: [학적]\n제목: [취업진로교육] 2023년 2학기 취업진로교육 대체이수 신청 접수 안내\n등록일: 2023-11-15\nlink: https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=469383&article.offset=0&articleLimit=500', metadata={'source': './data/school_notice.csv', 'row': 1}),
 Document(page_content='번호: 5828\n카테고리: [장학]\n제목: 2023년도 하반기 강화군 대학생 등록금 지원사업 안내\n등록일: 2023-11-14\nlink: https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=469304&article.offset=0&articleLimit=500', metadata={'source': './data/school_notice.csv', 'row': 2}),
 Document(page_content='번호: 5827\n카테고리: [장학]\n제목: [근로]2023학년도 1학기 1차 국가근로장학금 학생신청 안내[2022.11.24.(목)~12.29.(목)]\n등록일: 2023-11-14\nlink: https://www.kumoh.ac.kr/ko/sub06_01_01_0